# Markowitz Minimum Variance Portfolio

The Markowitz Minimum Variance Portfolio is a fundamental concept in modern portfolio theory, introduced by Harry Markowitz in 1952. The goal of this portfolio is to minimize the overall risk (or variance) for a given set of assets, without considering the expected return.

## Key Concepts

- **Portfolio Variance:** The measure of risk in a portfolio, defined as the weighted sum of the variances of individual assets and the covariances between them. It's calculated as:

  $$
  \sigma_p^2 = \sum_{i=1}^n \sum_{j=1}^n w_i w_j \sigma_{ij}
  $$

  Where:
  - $\sigma_p^2$ is the portfolio variance.
  - $w_i $ and $w_j$ are the weights of assets $i$ and $j$ in the portfolio.
  - $\sigma_{ij}$ is the covariance between assets $i$ and $j$.

- **Minimum Variance Portfolio:** Among all possible portfolios, this portfolio has the lowest possible variance. It is found by solving an optimization problem that minimizes portfolio variance subject to the constraint that the sum of the asset weights equals one:

  $$
  \min \sigma_p^2 \quad \text{subject to} \quad \sum_{i=1}^n w_i = 1
  $$

## Characteristics

- **Risk Minimization:** The primary focus is on minimizing risk rather than maximizing return. This makes it particularly attractive for risk-averse investors.
  
- **Diversification:** By combining assets that are not perfectly correlated, the portfolio reduces unsystematic risk through diversification.

- **Efficient Frontier:** The minimum variance portfolio is one of the portfolios on the efficient frontier, which represents the set of optimal portfolios offering the highest expected return for a given level of risk.

## Limitations

- **Assumptions:** The model assumes that asset returns are normally distributed and that past performance can predict future returns, which may not always hold true.
  
- **Data Sensitivity:** The results are highly sensitive to the input data, particularly the covariance matrix, which can lead to estimation errors.

Despite its limitations, the Markowitz Minimum Variance Portfolio remains a cornerstone of portfolio management and a valuable tool for risk management.

## Usage example

An usage example of strategy based on the markowitz min variance portfolio.

Your task is to propose a particular strategy similar to the one located at the folder `strategy`. 

In [ ]:
import pandas as pd
import quantstats as qs

## Load Data

You may organize the data the way you want. Here, we downloaded all data into the `dataset` directory in parquet format.

We also use auxiliar functions from the `data_market` directory to handle these data.

In [ ]:
from data_market.datalake import load_data

dict_data = load_data()

# Let's check the keys of the dictionary, each one a DataFrame
print(dict_data.keys())

In [ ]:
# Let's check the first DataFrame: prices of US stocks
dict_data['stocks']

## Strategy execution

Following, we test the execution of our strategy: for just a single day.

In [ ]:
# You must write all the code for your strategy entirely in the strategy directory
# Your strategy must implement the StrategyInterface interface defined in simulator/strategy_interface.py
from strategy.strategy_minRisk import MinRiskStrategy

strategy = MinRiskStrategy()

In [ ]:
# Execution for a single day
weights = strategy.calculate_next_weights(data = dict_data, t = 500, size = 20)

# Check if the returned DataFrame has the correct format
assert strategy.check_return(weights)

Your strategy must return a DataFrame with the columns: `date`, `ticker` and `weights`.

In [ ]:
weights

## Simulation

Following we simulate our strategy and generate the final report.

In [ ]:
from simulator.strategy_simulator import strategy_simulator

# Initialize data structures to store results
ret_port = pd.Series(dtype="float64")  # Series to store portfolio returns
weights_db = pd.DataFrame()  # DataFrame to store historical weights

# Loop through a range of time values
for t in range(500, len(dict_data['stocks'].index) - 1):

    # Use the strategy simulator to get portfolio's historical weights [weights_db]
    # and its next day returns [ret_port]
    ret_port, weights_db = strategy_simulator(path = "results/",
                                              strategy = strategy,
                                              data = dict_data,
                                              t = t,
                                              ret_port = ret_port,
                                              weights_db = weights_db,
                                              size = 10)

## Report

We can use the results of the simulation, saved in the directory `results`, to generate a report of our strategy using `quantstats`.

The simulation generate two parquet files:

- [ret_port.parquet](results/ret_port.parquet): DataFrame with the return of the portfolio, for each simulated datetime
- [wegiths_db.parquet](results/weights_db.parquet): DataFrame with the weights of each stock in the portfolio, for each simulated datetime

In [ ]:
ret_port = pd.read_parquet("results/ret_port.parquet")
ret_port['date'] = pd.to_datetime(ret_port['date'], format = "%Y-%m-%d")
ret_port = ret_port.set_index("date").ret_port

In [ ]:
ret_port.head(3)

Following we generate a HTML report, comparing our strategy with the SP500.

In [ ]:
qs.reports.html(ret_port, "SPY", text_description="""
    <p>Demonstration of a simple strategy.</p>
    <p><strong>Important</strong>: Transaction costs are not taken into account.</p>""")